In [64]:
import pandas as pd
import numpy as np
import os
from Data_Handler.DataReader import DataReader
from collections import Counter

In [67]:
f = open("Dataset/impressions_ICM.csv", "w+")
f.write("ItemID,FeatureID,Data\n")

"""
Return a dictionary of dictionaries. For each UserID there is a dictionary of ItemIDs as keys and a number, corresponding to 
how many times that ItemID has been presented to the given UserID, as values.

Args:
    target (int): UserIDs on which count ItemIDs presentations occurences
    items (numpy.array): ItemIDs on which count presentations occurences
Returns:
    dict: dictionary of dictionaries, for instance { user0:{item0:2, item1:23}, user1:{item2:11, item4:3} }
"""

df = pd.read_csv(filepath_or_buffer=os.getenv('INTERACTIONS_AND_IMPRESSIONS_PATH'),
                    sep=',',
                    names=[
    'UserID', 'ItemID', 'Impressions', 'Data'],
    header=0,
    dtype={'UserID': np.int32, 'ItemID': np.int32, 'Impressions': np.object0, 'Data': np.int32})
items = df['ItemID'].unique()
df = df.drop(['UserID'], axis=1)
df = df.drop(['Data'], axis=1)
df = df.dropna()
# add a comma at the end of each impression string in order to concat properly then
df['Impressions'] = df['Impressions'].apply(lambda x: str(x)+',')
df = df.groupby(['ItemID'], as_index=False)
# to concat impressions of each user
impressions_per_item = df['Impressions'].apply(sum)



max_for_each_item=[]
for item in items:
    impressions = impressions_per_item[impressions_per_item['ItemID']== item]['Impressions']
    if(impressions.size>0):
        # Get impressions string for the item in the loop
        impressions= impressions.iloc[0]
        # Split the impressions strings
        impressions= impressions.split(",")
        # Remove last element which is a '' due to last ','
        impressions= impressions[:-1]
        # Convert into list of integers
        impressions = [int(x) for x in impressions]
        # Remove item itself because it is present in every list impressions
        while(item in impressions):
            impressions.remove(item)

        # Count the impressions occurences
        counts = Counter(impressions)
        # Get the one most common
        most_common = counts.most_common(1)
        # Get the item occured the most and how many times, if it exists
        if(len(most_common)>0):
            impression_that_occurred_the_most, impression_occurences = most_common[0]
            # Append to a list contating all max occurences for each item having impression lists
            max_for_each_item.append(impression_occurences)

# Get the max of max occurences of all items having impressions lists
max_occurences = max(max_for_each_item)
print(max_occurences)

# For padding purposes
#import copy
#remaining_items = copy.deepcopy(items)


# Create file
for item in items:
    impressions = impressions_per_item[impressions_per_item['ItemID']== item]['Impressions']
    if(impressions.size>0):
        # Get impressions string for the item in the loop
        impressions= impressions.iloc[0]
        # Split the impressions strings
        impressions= impressions.split(",")
        # Remove last element which is a '' due to last ','
        impressions= impressions[:-1]
        # Convert into list of integers
        impressions = [int(x) for x in impressions]
        # Remove item itself because it is present in every list impressions
        while(item in impressions):
            impressions.remove(item)

        # Count impressions occurences for the item in the loop
        counts = Counter(impressions)

        # Remove duplicates before inserting into the file
        for element in impressions:
            while impressions.count(element) > 1:
                impressions.remove(element)

        # For each item and for each impression associated with
        # Insert a line in the file with the number of impression occurences weighted by the maximum number of an impression occurence
        for item_impression in impressions:
            impressions_count_per_item = counts[item_impression]
            f.write(f"{item},{item_impression},{impressions_count_per_item/max_occurences}\n")

            # For padding purposes
            #if(item in remaining_items):
            #    remaining_items.remove(item)

'''
# For padding purposes
for item in remaining_items:
    f.write(f"{item},{1},{0}\n")
    '''

7532


'\n# For padding purposes\nfor item in remaining_items:\n    f.write(f"{item},{1},{0}\n")\n    '